In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D, BatchNormalization, Multiply, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# paths
from google.colab import drive
drive.mount('/content/drive')
mri_folder = '/content/drive/MyDrive/MRI/mri'
mask_folder = '/content/drive/MyDrive/MRI/masks'

Mounted at /content/drive


In [ ]:
def load_images_and_masks(mri_folder, mask_folder, img_size=(256, 256)):
    mri_images = []
    mask_images = []

    for mri_filename in os.listdir(mri_folder):
        if mri_filename.endswith('.jpg'):
            mri_path = os.path.join(mri_folder, mri_filename)
            mask_path = os.path.join(mask_folder, mri_filename.replace('.jpg', '.png'))


            mri_img = load_img(mri_path, color_mode='grayscale', target_size=img_size)
            mri_img = img_to_array(mri_img) / 255.0


            mask_img = load_img(mask_path, color_mode='grayscale', target_size=img_size)
            mask_img = img_to_array(mask_img) / 255.0


            mask_img = (mask_img > 0.5).astype(np.float32)

            mri_images.append(mri_img)
            mask_images.append(mask_img)

    return np.array(mri_images), np.array(mask_images)

In [ ]:

mri_images, mask_images = load_images_and_masks(mri_folder, mask_folder)


X_train, X_val, y_train, y_val = train_test_split(mri_images, mask_images, test_size=0.2, random_state=42)

In [ ]:
def attention_gate(F_g, F_l, inter_channels):
    """
    F_g: result of upsampled layer
    F_l: Feature map from the skip connection.
    inter_channels: Intermediate number of channels.
    """
    # Transforming result of upsampled layer
    '''
    here, the simple difference between F_g and W_g is that we are reducing the number of channels in W_g for faster
    calculations and less computational load. say, if f_g had 512 channels, W_g will have 256 channels.
    '''
    W_g = Conv2D(inter_channels, kernel_size=1, strides=1, padding='same')(F_g)
    W_g = BatchNormalization()(W_g)

    # Transforming feature map
    W_x = Conv2D(inter_channels, kernel_size=1, strides=1, padding='same')(F_l)
    W_x = BatchNormalization()(W_x)

    # Adding the transformations, using relu to keep only the positive values
    psi = Activation('relu')(W_g + W_x)


    psi = Conv2D(1, kernel_size=1, strides=1, padding='same')(psi) #reducing the psi channels to 1
    psi = Activation('sigmoid')(psi) #converting values between 0s and 1s

    # Applying attention gate to the feature map. psi is the map having only relevant features, and hence, when applied to
    #the F_l, it will give us only the relevant features out of it.
    return Multiply()([F_l, psi])

In [ ]:
def attention_unet(input_size=(256, 256, 1)):
    inputs = Input(input_size)

    # Downsample
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Upsample with Attention Gates
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    att6 = attention_gate(up6, drop4, inter_channels=512)
    merge6 = concatenate([att6, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    att7 = attention_gate(up7, conv3, inter_channels=256)
    merge7 = concatenate([att7, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    att8 = attention_gate(up8, conv2, inter_channels=128)
    merge8 = concatenate([att8, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    att9 = attention_gate(up9, conv1, inter_channels=64)
    merge9 = concatenate([att9, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs, outputs)
    return model


model = attention_unet()

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Dice Score Metric
def dice_score(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(tf.round(y_pred))  # Round to get binary values
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# PPV (Precision / Positive Predictive Value)
def ppv(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(tf.round(y_pred))
    true_positives = K.sum(y_true_f * y_pred_f)
    predicted_positives = K.sum(y_pred_f)
    return (true_positives + smooth) / (predicted_positives + smooth)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.MeanIoU(num_classes=2),
        dice_score,
        ppv
    ]
)


In [ ]:
checkpoint = ModelCheckpoint('attention_unet_bce.h5', monitor='val_mean_io_u', mode='max', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_mean_io_u', mode='max', patience=10, restore_best_weights=True)

In [ ]:

#class_weights = {0: 1.0, 1: 10.0}
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    batch_size=8,
                    #class_weight=class_weights,
                    callbacks=[checkpoint, early_stopping])

Epoch 1/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 657ms/step - accuracy: 0.9976 - dice_score: 0.6503 - loss: 0.5118 - mean_io_u: 0.9107 - ppv: 0.5018

60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 830ms/step - accuracy: 0.9976 - dice_score: 0.6506 - loss: 0.5117 - mean_io_u: 0.9108 - ppv: 0.5022 - val_accuracy: 0.9977 - val_dice_score: 0.6558 - val_loss: 0.4885 - val_mean_io_u: 0.8996 - val_ppv: 0.5008
Epoch 2/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 650ms/step - accuracy: 0.9976 - dice_score: 0.6634 - loss: 0.5085 - mean_io_u: 0.9158 - ppv: 0.5210

60/60 ━━━━━━━━━━━━━━━━━━━━ 77s 748ms/step - accuracy: 0.9976 - dice_score: 0.6635 - loss: 0.5085 - mean_io_u: 0.9158 - ppv: 0.5211 - val_accuracy: 0.9975 - val_dice_score: 0.6398 - val_loss: 0.4866 - val_mean_io_u: 0.8997 - val_ppv: 0.4826
Epoch 3/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 79s 703ms/step - accuracy: 0.9978 - dice_score: 0.6833 - loss: 0.5009 - mean_io_u: 0.9205 - ppv: 0.5398 - val_accuracy: 0.9979 - val_dice_score: 0.6741 - val_loss: 0.4764 - val_mean_io_u: 0.8990 - val_ppv: 0.5233
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 82s 704ms/step - accuracy: 0.9981 - dice_score: 0.6986 - loss: 0.5004 - mean_io_u: 0.9204 - ppv: 0.5570 - val_accuracy: 0.9978 - val_dice_score: 0.6700 - val_loss: 0.4755 - val_mean_io_u: 0.8920 - val_ppv: 0.5197
Epoch 5/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 650ms/step - accuracy: 0.9981 - dice_score: 0.7077 - loss: 0.4943 - mean_io_u: 0.9247 - ppv: 0.5653

60/60 ━━━━━━━━━━━━━━━━━━━━ 86s 765ms/step - accuracy: 0.9981 - dice_score: 0.7076 - loss: 0.4943 - mean_io_u: 0.9246 - ppv: 0.5653 - val_accuracy: 0.9979 - val_dice_score: 0.6767 - val_loss: 0.4765 - val_mean_io_u: 0.9049 - val_ppv: 0.5225
Epoch 6/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 652ms/step - accuracy: 0.9981 - dice_score: 0.7017 - loss: 0.4935 - mean_io_u: 0.9182 - ppv: 0.5601

60/60 ━━━━━━━━━━━━━━━━━━━━ 84s 800ms/step - accuracy: 0.9981 - dice_score: 0.7020 - loss: 0.4934 - mean_io_u: 0.9182 - ppv: 0.5604 - val_accuracy: 0.9979 - val_dice_score: 0.6854 - val_loss: 0.4719 - val_mean_io_u: 0.9078 - val_ppv: 0.5332
Epoch 7/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 78s 737ms/step - accuracy: 0.9983 - dice_score: 0.7264 - loss: 0.4885 - mean_io_u: 0.9252 - ppv: 0.5904 - val_accuracy: 0.9977 - val_dice_score: 0.6635 - val_loss: 0.4635 - val_mean_io_u: 0.9031 - val_ppv: 0.5091
Epoch 8/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 633ms/step - accuracy: 0.9981 - dice_score: 0.7136 - loss: 0.4839 - mean_io_u: 0.9259 - ppv: 0.5740

...

60/60 ━━━━━━━━━━━━━━━━━━━━ 88s 843ms/step - accuracy: 0.9993 - dice_score: 0.8768 - loss: 0.3511 - mean_io_u: 0.9521 - ppv: 0.8061 - val_accuracy: 0.9990 - val_dice_score: 0.8085 - val_loss: 0.3448 - val_mean_io_u: 0.9256 - val_ppv: 0.6969
Epoch 47/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 650ms/step - accuracy: 0.9993 - dice_score: 0.8602 - loss: 0.3500 - mean_io_u: 0.9472 - ppv: 0.7877

60/60 ━━━━━━━━━━━━━━━━━━━━ 82s 838ms/step - accuracy: 0.9993 - dice_score: 0.8603 - loss: 0.3499 - mean_io_u: 0.9472 - ppv: 0.7878 - val_accuracy: 0.9990 - val_dice_score: 0.8116 - val_loss: 0.3443 - val_mean_io_u: 0.9259 - val_ppv: 0.6987
Epoch 48/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 644ms/step - accuracy: 0.9993 - dice_score: 0.8730 - loss: 0.3459 - mean_io_u: 0.9512 - ppv: 0.8033

60/60 ━━━━━━━━━━━━━━━━━━━━ 77s 758ms/step - accuracy: 0.9993 - dice_score: 0.8730 - loss: 0.3459 - mean_io_u: 0.9512 - ppv: 0.8032 - val_accuracy: 0.9988 - val_dice_score: 0.7808 - val_loss: 0.3433 - val_mean_io_u: 0.9260 - val_ppv: 0.6538
Epoch 49/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 652ms/step - accuracy: 0.9993 - dice_score: 0.8774 - loss: 0.3412 - mean_io_u: 0.9558 - ppv: 0.8067

60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 771ms/step - accuracy: 0.9993 - dice_score: 0.8773 - loss: 0.3412 - mean_io_u: 0.9557 - ppv: 0.8067 - val_accuracy: 0.9990 - val_dice_score: 0.8066 - val_loss: 0.3386 - val_mean_io_u: 0.9263 - val_ppv: 0.6926
Epoch 50/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 41s 683ms/step - accuracy: 0.9993 - dice_score: 0.8629 - loss: 0.3397 - mean_io_u: 0.9512 - ppv: 0.7819 - val_accuracy: 0.9991 - val_dice_score: 0.8311 - val_loss: 0.3314 - val_mean_io_u: 0.9256 - val_ppv: 0.7330
Epoch 51/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 653ms/step - accuracy: 0.9994 - dice_score: 0.8761 - loss: 0.3379 - mean_io_u: 0.9556 - ppv: 0.8072

60/60 ━━━━━━━━━━━━━━━━━━━━ 90s 824ms/step - accuracy: 0.9994 - dice_score: 0.8761 - loss: 0.3378 - mean_io_u: 0.9556 - ppv: 0.8072 - val_accuracy: 0.9991 - val_dice_score: 0.8219 - val_loss: 0.3287 - val_mean_io_u: 0.9280 - val_ppv: 0.7165
Epoch 52/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 76s 732ms/step - accuracy: 0.9994 - dice_score: 0.8807 - loss: 0.3310 - mean_io_u: 0.9562 - ppv: 0.8172 - val_accuracy: 0.9991 - val_dice_score: 0.8226 - val_loss: 0.3278 - val_mean_io_u: 0.9245 - val_ppv: 0.7214
Epoch 53/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 82s 738ms/step - accuracy: 0.9993 - dice_score: 0.8748 - loss: 0.3290 - mean_io_u: 0.9508 - ppv: 0.8136 - val_accuracy: 0.9990 - val_dice_score: 0.8049 - val_loss: 0.3256 - val_mean_io_u: 0.9253 - val_ppv: 0.6921
Epoch 54/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 41s 682ms/step - accuracy: 0.9993 - dice_score: 0.8726 - loss: 0.3275 - mean_io_u: 0.9509 - ppv: 0.8069 - val_accuracy: 0.9988 - val_dice_score: 0.7853 - val_loss: 0.3253 - val_mean_io_u: 0.9212 - val_ppv: 0.6593
E

In [ ]:
loss, accuracy, mean_iou, dice, precision = model.evaluate(X_val, y_val)

print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')
print(f'Validation Mean IoU: {mean_iou}')
print(f'Validation Dice Score: {dice}')
print(f'Validation PPV (Precision): {precision}')



4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 713ms/step - accuracy: 0.9991 - dice_score: 0.8230 - loss: 0.3289 - mean_io_u: 0.9262 - ppv: 0.7180
Validation Loss: 0.32872292399406433
Validation Accuracy: 0.9990703463554382
Validation Mean IoU: 0.9280261993408203
Validation Dice Score: 0.8252043128013611
Validation PPV (Precision): 0.7213262915611267
